

# 1. MIO problem — 4 examples

The MIO problem can be solved by benders decomposition. The MIO problem is used in 4 examples: Random Forest Regressor, Random Forest Classifier, Gradient Boosting Classifier and Gradient Boosting Regressor. The model is as follows：

<img style="float: left;" src="MIO.png" width=600 height=600>

---
## 1. Setup


### 1.1 Packages



In [1]:
#import sys
#sys.path.append("/Users/zhaoziyue/opt/anaconda3/lib/python3.8/site-packages")
import pandas as pd
from pandas import DataFrame
import scipy.stats
from scipy.stats import norm
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import *
from prepareziyue import *
from ziyuesolver_2 import *
from gurobipy import *

---
## 1. Random forests regressor

The first step is to prepare data. The data for this project comes from [UCI machine learning repository](http://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength). Concrete is the most important material in civil engineering. The concrete compressive strength is a highly nonlinear function of age and ingredients. These data contain columns for  `Cement`, `BlastFurnaceSlag`, `FlyAsh`, `Water`, `Superplasticizer`, `CoarseAggregate`, `FineAggregate`, `Age` and `CompressiveStrength`. 

These data are available in `concrete.csv`.


In [2]:
df = pd.read_csv('concrete.csv')
df=df.drop(df.columns[0], axis=1)
X_variable=df.iloc[:,:-1]
Y_variable=df.iloc[:,-1]

labels=np.array(Y_variable)
feature_list = list(X_variable.columns)
features = np.array(X_variable)

In [3]:
df

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


We use the random forests regressor.

In [6]:
flag=1 # Random forests regressor (n_estimator=2)
rf=get_rf_gb_(features,labels,2,flag)
trees=get_input(rf,flag)

We define the domain.

In [7]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

We run the MIP_solver, which is the Benders Decomposition method and gurobi solver.

In [8]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 911 rows, 927 columns and 1820 nonzeros
Model fingerprint: 0xbb10eac9
Model has 2414 general constraints
Variable types: 10 continuous, 917 integer (917 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [8e+01, 8e+01]
  GenCon rhs range [9e-01, 1e+03]
  GenCon coe range [1e+00, 1e+00]
Presolve added 1081 rows and 1079 columns
Presolve time: 0.16s
Presolved: 1992 rows, 2006 columns, 4600 nonzeros
Presolved model has 634 SOS constraint(s)
Variable types: 1534 continuous, 472 integer (472 binary)
Find a violated constraint!

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.1400000e+01   2.930972e+04   0.000000e+00     27s
    1285    7.1180000

For the maximization problem, we find the lower bound 51.56 and the upper bound 54.46. Gap is 5.62%. The model time is 350s.

---
## 2. Gradient Boosting Regressor

We also use `concrete.csv`.

In [9]:
flag=4 # Gradient Boosting Regressor (n_estimator=100)
gb=get_rf_gb_(features,labels,100,flag)
trees=get_input(gb,flag)

In [11]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 379 rows, 395 columns and 658 nonzeros
Model fingerprint: 0x06195272
Model has 1348 general constraints
Variable types: 108 continuous, 287 integer (287 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [5e-01, 3e+01]
  GenCon rhs range [9e-01, 1e+03]
  GenCon coe range [1e+00, 1e+00]
Presolve added 842 rows and 938 columns
Presolve time: 0.08s
Presolved: 1221 rows, 1333 columns, 2899 nonzeros
Presolved model has 482 SOS constraint(s)
Variable types: 1023 continuous, 310 integer (310 binary)
Find a violated constraint!

Root relaxation: objective 1.008752e+01, 839 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

Find a violated constraint!
 11963  3144    1.83304    7    -          -    4.88620      -  11.7  341s
Find a violated constraint!
 12208  3222    1.67644    7    -          -    4.80649      -  11.7  345s
Find a violated constraint!
 12452  3297    1.54179    7    -          -    4.75482      -  11.7  352s
Find a violated constraint!
 12697  3376    1.20231    7    -          -    4.64360      -  11.8  361s
*12934  3468               7       0.8223561    4.43867   440%  11.8  366s
Find a violated constraint!
*12943  3471               8       0.9268742    4.43867   379%  11.8  377s
Find a violated constraint!
Find a violated constraint!
 13188  3367     cutoff   13         0.92687    4.26243   360%  11.8  387s
Find a violated constraint!
*13327  3378               9       0.9433424    4.03311   328%  11.8  394s
 13855  2688     cutoff    9         0.94334    3.42698   263%  12.1  395s
Find a violated constraint!
Find a violated constraint!
Find a violated constraint!
 14606  2046    1

For the maximization problem, we find the lower bound 0.94334  and the upper bound 1.08499. Gap is 15.0%. The model time is 416s.

---
## 3. Gradient Boosting Classifier

The first step is also to prepare data. We can generate our dataset.

In [16]:
# Generate some data
x=np.linspace(0,1,100)
z=scipy.stats.norm.rvs(loc=0,size=100,scale=1)
y1=x**2
y=x**2+z
df=DataFrame({"x":x,"y":y,"class":z})
df["class"][df["class"]>=0]=1
df["class"][df["class"]<0]=-1
xx=df[["x","y"]]
yy=df["class"]

In [17]:
df

,x,y,class
0,0.000000,0.177703,1.0
1,0.010101,-0.454623,-1.0
2,0.020202,2.324861,1.0
3,0.030303,-1.616880,-1.0
4,0.040404,1.635383,1.0
...,...,...,...
95,0.959596,1.122034,1.0
96,0.969697,1.024990,1.0
97,0.979798,0.780589,-1.0
98,0.989899,-0.855687,-1.0


In [7]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

In [18]:
flag=3 # Gradient Boosting Classifier tree
gb_3=get_rf_gb_(xx,yy,100,flag)
trees_3=get_input(gb_3,flag)

In [21]:
MIP_solver(trees_3,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 125 rows, 129 columns and 150 nonzeros
Model fingerprint: 0x2bbcecab
Model has 1242 general constraints
Variable types: 102 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [0e+00, 0e+00]
  GenCon rhs range [4e-02, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 1755 rows and 1855 columns
Presolve time: 0.05s
Presolved: 1880 rows, 1984 columns, 4889 nonzeros
Presolved model has 1242 SOS constraint(s)
Variable types: 1344 continuous, 640 integer (640 binary)
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 5.18 seconds (0.02 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: -0 
No other

Why is it? My opinion is as follows: The Gradient Boosting Classifier trees set have many regression trees. And the value of leafs are real number. It is not the prediction of the leaf. So it may not be suitable for the tree ensemble optimization model.

In [ ]:
# We could check the trees in Gradient Boosting Classifier.
y_pred=gb_3.predict(xx)
print(tree.export_text(trees[0]))
fig,ax=plt.subplots(figsize=(20,20))
tree.plot_tree(trees[1], filled=True)

## 4. Random Forest Classifier

#### 4.1 Generating classification data
We use the data we generated for section 3. Gradient Boosting Classifier.

In [24]:
flag=2 # Random Forest Classifier
rf_4=get_rf_gb_(xx,yy,100,flag)
trees=get_input(rf_4,flag)

In [ ]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

In [26]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 340 rows, 344 columns and 580 nonzeros
Model fingerprint: 0xca3b46cc
Model has 1454 general constraints
Variable types: 102 continuous, 242 integer (242 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [3e-03, 2e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 2081 rows and 2181 columns
Presolve time: 0.51s
Presolved: 2421 rows, 2525 columns, 6287 nonzeros
Presolved model has 1454 SOS constraint(s)
Variable types: 1556 continuous, 969 integer (969 binary)

Root relaxation: objective 1.000000e+00, 1521 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

My opinion: The reason of this phenomenon may be that the problem is a binary classification problem. It is easy to get the region that has a prediction of 1 for each tree. And the trees are independent but sometimes similar. So it is easy to get the maximum of the tree ensemble problem.

#### 4.2 Using iris data
So we use other classification dataset. The following data are iris data. And here are three classes in the iris data.

In [18]:
from sklearn.datasets import load_iris
iris=load_iris()
X=iris["data"]
Y=iris["target"]

We also use Random Forest Classifier.

In [19]:
# Random Forest Classifier
flag=2 
rf_5=get_rf_gb_(X,Y,100,flag)
trees=get_input(rf_5,flag)

In [20]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

In [21]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 208 rows, 216 columns and 316 nonzeros
Model fingerprint: 0x48e4613f
Model has 1558 general constraints
Variable types: 104 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [2e+00, 2e+00]
  GenCon rhs range [7e-01, 7e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 2235 rows and 2335 columns
Presolve time: 0.12s
Presolved: 2443 rows, 2551 columns, 6422 nonzeros
Presolved model has 1558 SOS constraint(s)
Variable types: 1662 continuous, 889 integer (889 binary)
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 4.99 seconds (0.06 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 2 

Optimal

#### 4.3 Using manh tickets data
We use other classification dataset. And here are five classes in the data.

In [2]:
manh_tickets = pd.read_csv("manh_ticket.csv")
X = manh_tickets[['lon','lat']]
y = manh_tickets.precinct

In [3]:
# Random Forest Classifier
flag=2 
rf_6=get_rf_gb_(X,y,20,flag)
trees=get_input(rf_6,flag)

In [4]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

In [5]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2000 rows, 2004 columns and 3980 nonzeros
Model fingerprint: 0x2c06a2e5
Model has 7382 general constraints
Variable types: 22 continuous, 1982 integer (1982 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [4e+00, 4e+00]
  GenCon rhs range [4e+01, 7e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve added 11053 rows and 11073 columns
Presolve time: 3.15s
Presolved: 13053 rows, 13077 columns, 28072 nonzeros
Presolved model has 7382 SOS constraint(s)
Variable types: 7404 continuous, 5673 integer (5673 binary)

Root relaxation: objective 4.000000e+00, 8824 iterations, 0.24 seconds (0.28 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

#### 4.3 Generating data with ten classes.
We generate other classification dataset with ten classes. 

In [6]:
from sklearn.datasets import make_classification
X,Y=make_classification(n_samples=150,n_classes=10,n_features=5,n_informative=5,n_redundant=0)

We also use Random forest Classifier.

In [7]:
# Random Forest Classifier
flag=2 
rf_7=get_rf_gb_(X,Y,20,flag)
trees=get_input(rf_7,flag)

In [8]:
lb={}
ub={}
for i in total_split_variable(trees):
    lb[i]=-100
    ub[i]=100

In [9]:
MIP_solver(trees,flag,lb,ub)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1006 rows, 1016 columns and 1992 nonzeros
Model fingerprint: 0xccaa8b6d
Model has 2246 general constraints
Variable types: 25 continuous, 991 integer (991 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [9e+00, 9e+00]
  GenCon rhs range [2e-03, 4e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 3349 rows and 3369 columns
Presolve time: 0.77s
Presolved: 4355 rows, 4385 columns, 10814 nonzeros
Presolved model has 2246 SOS constraint(s)
Variable types: 2271 continuous, 2114 integer (2114 binary)

Root relaxation: objective 9.000000e+00, 2638 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In